# Chapter 6. 이미지 변환
이미지 변형, 이미지 변환 -> 데이터 전처리 과정에서 주로 활용  

이미지 변형은 이미지 해석을 위해 조금 더 검출하기 쉬운 데이터로 만드는 과정  

이미지 변환은 이미지 데이터의 개수를 늘리거나 줄여 알고리즘의 연산량을 줄이는 것이 목표  



## 이미지 변환  분류

![image](https://user-images.githubusercontent.com/84179578/139629247-69369992-d9c6-4306-8028-ba5043a7b799.png)

- __강체 변환 (Rigid Transformation)__
  - 유클리디언 변환 (Euclidean Transformation)
  - 변환의 기준점으로부터 크기와 각도가 보존되는 변환
  - 평행이동(translation), 회전(rotation)  
  

- __유사 변환 (Similarity Transformation)__
  - 강체 변환에서 등방성 (Isotropic) 크기 변환 추가
  - 평행이동(translation), 회전(rotation), 크기 변환 (scailing)


- __선형 변환 (Linear Transformation)__
  - 벡터 공간에서의 이동
  - 크기 변환(scailing), 반사(reflection), 기울임(shear)
  - 이방성 (Anisotropic) 가질 수 있음

> _등방성 (isotropic) <-> 이방성 (anisotropic)_   
>
> 
>![image](https://user-images.githubusercontent.com/84179578/139630764-2508bf49-6637-4244-8018-c954471551e6.png)  
>
> [_영상처리에서 등방성 이미지, 이방성 이미지 란?_](https://bskyvision.com/778)


- __아핀 변환 (Affine Transformation)__
  - 선형 변환에 이동 변환까지 포함된 변환
  - 선의 수평성 유지 -> 변환 전의 서로 평행한 선은 변환 후에도 평행
  - ex) 직사각형 -> 평행사변형
  

- __원근 변환 (Perspective Transformation)__
  - 원근감을 표현하기 위한 변환
  - 선의 수평성 유지 x
  - ex) 직사각형 -> 임의의 사각형

## 1. 확대 & 축소

알고리즘에서 요구하는 입력 이미지의 크기 -> 이미지 크기 변환 필요

이미지 확대, 축소는 __이미지 피라미드 (image pyramid)__ 를 활용해 원하는 단계까지 샘플링하는 작업

![image](https://user-images.githubusercontent.com/84179578/139654866-02b751db-2017-4e7f-8ea2-5126a6b37746.png)

- 업샘플링 : 원본 이미지에서 크기를 확대하는 것, 하위 단계의 이미지를 생성
- 다운샘플링 : 원본 이미지에서 크기를 축소하는 것, 상위 단계의 이미지를 생성  

#### 이미지 피라미드

- __가우시안 피라미드 (Gaussian Pyramid)__ 
  - 업샘플링은 입력 이미지에 새로운 행과 열을 추가해 짝수행과 짝수열을 만든 후 하위 단계 이미지 생성 (M x N -> 2M x 2N 이미지 생성)
  - 다운샘플링은 입력 이미지에 짝수행과 짝수열을 제거하여 상위 단계 이미지 생성 (M x N -> M/2 x N/2 이미지 생성)  
  - 단계마다 면적차이는 4배
  - ![image](https://user-images.githubusercontent.com/84179578/139658242-d0e6ea72-cadc-49af-8ec6-f8d0680e3163.png)
  

- __라플라시안 피라미드 (Laplacian pyramid)__
  - 라플라시안 피라미드는 가우시안 피라미드 이미지에서 업샘플링된 이미지를 감산해서 가우시안 피라미드의 차이를 계산
  - $$ G_0 = Image$$
  $$G_{i+1} = Down(G_i)$$
  $$L_i = G_i - Up(G_{i+1})$$
  - 피라미드의 각 레벨에서 경계선 이미지를 얻을 수 있음
 
  - ![image](https://user-images.githubusercontent.com/84179578/139658189-cb47197b-0f70-486e-85e0-69dd7bcdcf2c.png)

### 이미지 확대 (Pyramid Up)

__Python OpenCV 의 이미지 확대 함수__

```python
dst = cv2.pyrUp(
    src,
    dstSize = None          
    borderType = Npne
)
```

입력 이미지 (`src`) 의 행과 열을 2배로 키워 이미지를 확대하는 변환 함수  

입력 이미지에 새로운 행과 열을 추가해 짝수 행과 짝수 열을 만들고 0 의 값을 할당 -> 이후 색상이 결정되지 않은 픽셀은 근삿값으로 채워 가우시안 필터로 컨벌루션 진행(블러 처리)

1. 입력 이미지에서 짝수 열과 행에 픽셀을 추가
2. 추가한 픽셀 값을 근삿값으로 채움
3. 가우시안 필터로 컨볼루션 진행 (이미지를 블러처리)하여  하위 단계 이미지 생성

이때 가우시안 필터는 평균 밝기로 복원하기 위해서 4의 값을 사용해 정규화 -> 다운샘플링시 사용하는 가우시안 필터에 4를 곱한것


여기서 `dstSize` 는 출력 이미지의 크기를 나타냄

`dstSize` 를 할당해서 사용할때 다음 조건을 충족해야함  
|$dstsize.width$ - $src.cols$ x 2 | <=  ($dstsize.width$ mod2)  
|$dstsize.width$ - $src.rows$ x 2 | <=  ($dstsize.height$ mod2)  
-> 결과 이미지(`dst`) 의 크기가 입력 이미지 (`src`) 크기의 약 2배 인것을 의미 

### 이미지 축소(Pyramid Down)
__Python OpenCV 의 이미지 축소 함수__

```python
dst = cv2.pyrDown(
    src,
    dstSize = None          
    borderType = Npne
)
```

입력 이미지 (`src`) 의 행과 열을 2배로 축소하여 이미지를 확대하는 변환 함수,

이미지에 흐림 효과를 적용한 후 다운샘플링 적용

1. 입력 이미지에 가우시안 필터로 컨볼루션 진행 (이미지 블러처리)
2. 짝수 행과 열을 제거하여 상위 단계 이미지 생성


여기서 `dstSize` 는 출력 이미지의 크기를 나타냄

`dstSize` 를 할당해서 사용할때 다음 조건을 충족해야함  
|$dstsize.width$ - $src.cols$ x 2 | <=  2  
|$dstsize.width$ - $src.rows$ x 2 | <=  2  
-> 결과 이미지(`dst`) 의 크기가 입력 이미지 (`src`) 크기의 약 1/2배 인것을 의미 


이때 사용하는 가우시안 필터
![image](https://user-images.githubusercontent.com/84179578/139663677-cc4d909c-b845-4180-8ad0-d16a1f00f70d.png)

### Python 에서 이미지 확대

In [10]:
import cv2

src = cv2.imread("img.jpg")
cv2.imshow("src", src)  # 입력 이미지
cv2.waitKey(0)
cv2.destroyAllWindows()

![image](https://user-images.githubusercontent.com/84179578/139665030-ae548c79-2243-42ab-a595-23205ca7166b.png)

In [11]:
dst = src.copy()

# 1번째 확대
dst = cv2.pyrUp(dst)

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

![image](https://user-images.githubusercontent.com/84179578/139665429-dd88ba34-c0f0-4783-9feb-e365c7dbf1b5.png)

In [12]:
# 2번째 확대
dst = cv2.pyrUp(dst)

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

![image](https://user-images.githubusercontent.com/84179578/139665509-4bb87de9-665f-4cb6-ac82-ef5341a3fd5e.png)

### Python 에서 이미지 축소

In [4]:
import cv2

src = cv2.imread("ferris-wheel.jpg")
cv2.imshow("src", src)  # 입력 이미지
cv2.waitKey(0)
cv2.destroyAllWindows()

#### 입력 이미지
![image](https://user-images.githubusercontent.com/84179578/139664079-0d52cf99-3a78-42d6-89af-951f57f49a90.png)

In [5]:
dst = src.copy()

# 1번째 축소
dst = cv2.pyrDown(dst)

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()


![image](https://user-images.githubusercontent.com/84179578/139664198-4f1edd44-026b-4d06-986d-cc728c27ef41.png)

In [6]:
# 2번째 축소
dst = cv2.pyrDown(dst)

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()


![image](https://user-images.githubusercontent.com/84179578/139664294-d9d74eb2-1669-4d58-885f-9c14f1a61042.png)

for 문을 이용하여 한번에 원하는 크기 만큼 확대 및 축소 시킬 수 있다.
```python
for i in range(3):
    dst = cv2.pyDown(dst)
```

## 2. 이미지 크기 조절

이미지 크기 조절 함수는 사용자가 원하는 크기로 이미지를 변환

이미지를 임의의 크기로 조절하는 방법
- 이미지의 크기를 사용자가 요구하는 절대 크기로 변경하는 것 -> ex) 640x480 -> 123x456 이미지로 변환
- 이미지의 크기를 비율에 맞게 상대적인 크기로 변경하는 것 -> 입력 이미지의 크기와 비례하도록 너비와 높이가 계산

#### Python OpenCV 에서 사용하는 이미지 크기 조절 함수
```python
dst = cv2.resize(
    src,
    dsize,
    fx = None,
    fy = None,
    interpolation = None
)
```

입력 이미지(`src`) 의 크기를 절대 크기(`dsize`) 나 상대 크기(`fx`, `fy`) 로 변환

상대 크기(`fx`, `fy`) 로 변환하기 위해서는 절대 크기(`dsize`)의 값을 (0,0) 으로 설정하고 `fx`, `fy`에 각각 x축과 y축에 적용할 비율 계수를 설정해야함

-> 만약 절대크기(`dsize`) 에 0 이 아닌 값이 포함돼 있으면 `fx`,`fy` 값이 할당 돼 있더라도 절대크기(`dsize`) 를 이용하여 이미지 크기를 조절함


`interpolation` 은 이미지의 크기를 조절할 때 사용할 보간법을 지정함
- `cv2.INTER_NEAREST` : 가장 가까운 이웃 보간법, 가장 가까운 원본 픽셀을 사용
- `cv2.INTER_LINEAR`, `cv2.INTER_LINEAR_EXACT` : 쌍 선형 보간법, 2x2 크기의 주변 원본 픽셀을 거리에 따라 선형적으로 가중치를 할당해서 사용
- `cv2.INTER_AREA` : 영역 보간법, 결과 이미지 픽셀을 입력 이미지의 픽셀 위치에 배치하고 겹치는 영역의 평균을 구해 사용
- `cv2.INTER_CUBIC` : 4x4 바이 큐빅 보간법, 4x4 크기의 주변 원본 픽셀을 3차 큐빅 스플라인으로 계산하여 하용
- `cv2.INTER_LANZOS4` :8x8 란초스 보간법, 8x8 크기의 주변 원본 픽셀을 계산하여 사용

기본적으로 쌍 선형 보간법을 가장 많이 활용  
이미지를 확대하는 경우 -> 주로 쌍 선형 보간법, 바이큐빅 보간법 사용  
이미지를 축소하는 경우 -> 주로 영역 보간법을 사용


### Python OpenCV 의 이미지 크기 조절

In [ ]:
import cv2

src = cv2.imread("car.png")

# 입력이미지
dst = src[280:310, 240:405]


cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

![image](https://user-images.githubusercontent.com/84179578/139667814-3c7ffb7e-30d0-4896-9e06-3f49f185e80e.png)

In [13]:
# 크기 변경 후 이미지

dst = cv2.resize(dst, dsize=(256, 256), interpolation=cv2.INTER_NEAREST)

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

![image](https://user-images.githubusercontent.com/84179578/139667447-8cb867d0-f4c7-41ce-8772-ba77fedb0feb.png)